# Intro

This notebook is used to cleanup the raw hBEC TIFF stacks. 

It applies a contrast correction process and rescales the pixel values from `uint16` to `uint8`  

## Imports

In [ ]:
import os
import numpy as np
import joblib
from skimage import exposure
from tqdm import tqdm

from fam13a import utils

## Setup constants

In [ ]:
PROJ_ROOT = utils.here()
RAW_ROOT = os.path.join(PROJ_ROOT, 'data', 'raw')
HBEC_ROOT = os.path.join(RAW_ROOT, '2020_11_16_all', 'hbec')
EXP_IDS = os.listdir(HBEC_ROOT)

OUTPUT_ROOT = os.path.join(PROJ_ROOT, 'data', 'interim', 'hbec')

NCPUS = 1

INPUT_FORMAT = 'tif'

In [ ]:
EXP_IDS

In [ ]:
# find all raw image stack files
raw_stack_files = []
for exp_id in EXP_IDS:
    d_root = os.path.join(HBEC_ROOT, exp_id)
    for root, _, files in os.walk(d_root):
        for _f in files:
            if _f.lower().endswith(INPUT_FORMAT) and not _f.lower().startswith('max') and not _f.lower().startswith('.'):
                raw_stack_files.append((exp_id, root, _f))

# Process

define a simple helper function to allow for parallelisation of processing across different TIFF stacks

In [ ]:
def process(raw_file):
    exp_id, *raw_file = raw_file
    frames = utils.frames_from_stack(os.path.join(*raw_file))
        
    # equalize the intensity in each frame, then rescale the values to 0-255 range
    frames = exposure.equalize_adapthist(frames, kernel_size=15)
    frames = exposure.rescale_intensity(frames, out_range=np.uint8)
    frames = np.stack(frames, axis=0)

    stack_out_file = raw_file[-1].replace(' ', '_')
    
    output_folder = os.path.join(OUTPUT_ROOT, exp_id)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    utils.frames_to_stack(os.path.join(OUTPUT_ROOT, exp_id, stack_out_file), frames)          

In [ ]:
# parallelize the processing over files. 
# NOTE: each file requires ~65GB of memory and ~5 minutes to process
# with NCPUS=4, the total time to run across 24 wells is 76 min
with joblib.Parallel(n_jobs=NCPUS, verbose=20) as par:
    
    par(joblib.delayed(process)(raw_file) for raw_file in raw_stack_files)        